# Comparison of conditional soil moisture statistics
## Objectives:
- Analyze if GPM rainfall data could provide useful information on the conditional soil moisture
- Does GPM capture temporal statistics (e.g., mean and variance) of the observed soil moisture?
- How much information does SMAP satellite-based soil moisture contain compared to in situ soil moisture observations?
- Validity: Are short-term SMAP data (e.g., 5-6 years) representative of the conditional soil moisture information from in situ observations?
- Overall, how good rainfall-driven model soil moisture is compared to SMAP soil moisture with reference to the in situ observations?

## Methods:

- Use GPM IMERG half-hourly precipitation forcing to simulate soil moisture


In [1]:
import pandas as pd
import numpy as np
# import datetime
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib
from datetime import datetime
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('font', family='default')
import os

In [4]:
# import GPM-gage date-matched data
# sm_site_list = pd.read_csv('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gages/ARS_data/SF_stations_.csv')
lut_gpm_gage = pd.read_csv('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/GPM/lut_gage_gpm_v1.csv')
fn_format = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gages/ARS_data/1607.zip/SMAP/{st_id}_20130101.csv'
##



In [5]:
# ARS gage data
header_list = ['ID','Yr','Mo','Day','Hr','Min','TOY','Tair','PREC_1','PREC_2','SM_5','ST_5','SM_10','ST_10','SM_20','ST_20','SM_50','ST_50']
dtypes = 6*['string'] + 12*['float']
dtype_dict = dict()
for i,col in enumerate(header_list):
    dtype_dict[col] = dtypes[i]

# import ARS data into sf_gage dictionary
# There is a 5 hours offset to make ARS gage data time-zone UTC (found from cross-correlation analysis)
sf_gages = dict()

for st_id in lut_gpm_gage['st_id']:
    st_id = str(st_id)
    fn = fn_format.format(st_id=st_id)
    sf_gages[st_id] = pd.read_csv(fn, index_col=False, skiprows=2)
    sf_gages[st_id].columns = header_list
    sf_gages[st_id] = sf_gages[st_id].astype(dtype_dict)
    sf_gages[st_id] = sf_gages[st_id].replace([-99, -88], np.nan)
    dt = sf_gages[st_id]['Yr'] + '-' + sf_gages[st_id]['Mo'] + '-' + sf_gages[st_id]['Day'] + ' ' + sf_gages[st_id]['Hr'] + ':0' + sf_gages[st_id]['Min']
    sf_gages[st_id]['date'] = pd.to_datetime(dt)
    sf_gages[st_id]['date'] = sf_gages[st_id]['date'] + pd.DateOffset(hours=5)

# GPM data processed with Google Earth Engine (fid_{fid}.csv)
# Create a GPM dataset for each pixel over the South Fork watershed
# Note: GPM Data are provided with a 30-min interval but values are in mm/hr rainfall rate
gage_gpm_fmt = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/GPM/ars/fid_{fid}.csv'
gpm_gage = dict()
for fid in range(20):
    fid = str(fid)
    _data = pd.read_csv(gage_gpm_fmt.format(fid=fid), index_col=None)
    _data['date'] = pd.to_datetime(_data['date'])
    x = _data.set_index('date')
    gpm_gage[fid] = x['value'].resample('H').sum()/2
    gpm_gage[fid] = gpm_gage[fid].reset_index()

In [6]:
# Save data in a pickle organized as:
gpm_gage_sf = dict()
gpm_gage_sf['gpm_gage'] = gpm_gage
gpm_gage_sf['sf_gages'] = sf_gages
gpm_gage_sf['lut_gpm_gage'] = lut_gpm_gage # Lookup table for gage-GPM 

# import pickle
# with open('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gpm_gage_sf.pickle', 'wb') as handle:
#     pickle.dump(gpm_gage_sf, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [4]:
# Match the timestamps for both datasets

gage_gpm_matched = dict()
for i, st_id in enumerate(lut_gpm_gage['st_id']):
    fid = str(lut_gpm_gage['FID'][i])
    st_id = str(st_id)
    gpm_gage_ = gpm_gage[fid]
    sf_gages_ = sf_gages[st_id]
    gage_gpm_matched[st_id] = pd.merge(gpm_gage_, sf_gages_)

In [5]:
# Summarize the precipitation totals from GPM and gages
gpm_gage_summary = dict()
gage_p_sum = []
for i, st_id in enumerate(lut_gpm_gage['st_id']):
    st_id = str(st_id)
    fid = lut_gpm_gage['FID'][i]
    gage_p_sum.append(( st_id,
                        fid,
                        np.nansum(gage_gpm_matched[st_id]['PREC_1'][~gage_gpm_matched[st_id]['PREC_1'].isna()]),    # continues 
                        np.nansum(gage_gpm_matched[st_id]['PREC_2'][~gage_gpm_matched[st_id]['PREC_2'].isna()]),    # continues 
                        np.nansum(gage_gpm_matched[st_id]['value'][~gage_gpm_matched[st_id]['PREC_1'].isna()]),     # continues 
                        np.nansum(gage_gpm_matched[st_id]['value'][~gage_gpm_matched[st_id]['PREC_2'].isna()]) 
                    ))

gage_p_sum = pd.DataFrame(gage_p_sum, columns=['st_id', 'FID', 'sum_PREC_1','sum_PREC_2', 'sum_GPM_1', 'sum_GPM_2'])


# Multi-scale comparisons (time)
## Compare precipitation data from GPM and Gage for three different time scales:
- Hourly
- Daily
- Monthly
- Whole record (2013-2020 Note:varying for some stations)

# Gage data availability plot

In [130]:
fig, ax = plt.subplots(1,1,figsize=(16, 10))
for i, st_id in enumerate(lut_gpm_gage['st_id']):
    st_id = str(st_id)
    idx = np.where(~gage_gpm_matched[st_id]['PREC_1'].isna())#~gage_gpm_matched[st_id]['PREC_1'].isna()
    y = [i+1]*len(idx[0])
    ax.scatter(gage_gpm_matched[st_id]['date'][idx[0]], y, alpha=0.3, c='black', s = 2)
ax.tick_params(labelsize = 20)
ax.set_yticks([x for x in range(1,21,2)])
ax.set_ylabel(r'Station ID', fontsize = 24)
# ax.set_xlabel(r'Initial Soil Moisture \ ($mm$)', fontsize = 24)
ax.grid('major',axis='x')
fig.savefig('/mnt/d/ubuntu/projects/gatechProjects/StochSM/figures/gage_availability.jpg', dpi=300, bbox_inches='tight')
plt.close(fig)

In [13]:
# Summarize the precipitation totals from GPM and gages
gpm_gage_summary = dict()
gage_p_sum = []
for i, st_id in enumerate(lut_gpm_gage['st_id']):
    st_id = str(st_id)
    fid = lut_gpm_gage['FID'][i]
    gage_p_sum.append(( st_id,
                        fid,
                        np.nansum(gage_gpm_matched[st_id]['PREC_1'][~gage_gpm_matched[st_id]['PREC_1'].isna()]),    # continues 
                        np.nansum(gage_gpm_matched[st_id]['PREC_2'][~gage_gpm_matched[st_id]['PREC_2'].isna()]),    # continues 
                        np.nansum(gage_gpm_matched[st_id]['value'][~gage_gpm_matched[st_id]['PREC_1'].isna()]),     # continues 
                        np.nansum(gage_gpm_matched[st_id]['value'][~gage_gpm_matched[st_id]['PREC_2'].isna()]) 
                    ))
gage_p_sum = pd.DataFrame(gage_p_sum, columns=['st_id', 'FID', 'sum_PREC_1','sum_PREC_2', 'sum_GPM_1', 'sum_GPM_2'])

In [48]:
# create precipitation timeseries for each time-scale
# Initialize a dictionary with keys for each time-scale
gpm_gage_mscale = dict()
for tscale in ['H','D', 'M']:
        gpm_gage_mscale[tscale] = dict()
# Hourly
# for i, st_id in enumerate(lut_gpm_gage['st_id']):
#         st_id = str(st_id)
#         gpm_gage_mscale['H'][st_id] = gage_gpm_matched[st_id]

# Daily and Monthly
# Accumulate the rainfall totals to each day where there is observation from the gage
for tscale in ['H', 'D', 'M']:
        for i, st_id in enumerate(lut_gpm_gage['st_id']):
                st_id = str(st_id)
                gpm_gage_mscale[tscale][st_id] = dict()
                for buck_no in ['PREC_1', 'PREC_2']:
                        if tscale=='H':
                                ts_ = gage_gpm_matched[st_id][['date','value',buck_no]][~gage_gpm_matched[st_id][buck_no].isna()]
                                ts_ = ts_.set_index('date')
                                # ts_ = ts_[['value','PREC_1','PREC_2']]
                                gpm_gage_mscale[tscale][st_id][buck_no] = ts_
                        else:        
                                ts_ = gage_gpm_matched[st_id][['date','value',buck_no]][~gage_gpm_matched[st_id][buck_no].isna()]
                                ts_ = ts_.set_index('date')
                                # ts_ = ts_[['value','PREC_1','PREC_2']]
                                gpm_gage_mscale[tscale][st_id][buck_no] = ts_.resample(tscale).agg(pd.Series.sum, min_count=2)

# Rainfall Analysis
## Compare Precipitation totals and analyze rainfall detection

In [154]:
tscale_meta = {
    'H':
        {
        'ticklabels': ['', '10', '20', '30', '40', ''],
        'ticks': [0, 10, 20, 30, 40, 50],
        'ax_offset': 2,
        'alpha': 0.2
        },    
    'D':
        {
        'ticklabels': ['', '20', '40', '60', '80', ''],
        'ticks': [0, 20, 40, 60, 80, 100],
        'ax_offset': 4,
        'alpha': 0.3
        },
    'M':
        {
        'ticklabels': ['', '100', '200', '300', '400', ''],
        'ticks': [0, 100, 200, 300, 400, 500],
        'ax_offset': 10,
        'alpha': 0.5
        }
}
for tscale in ['H', 'D', 'M']:        
    fig, ax = plt.subplots(3, 5, figsize = (25, 14), sharex = True, sharey = True)
    for i, st_id in enumerate(lut_gpm_gage['st_id'][0:15]):
        st_id = str(st_id)
        i_col = i % 5
        i_row = int((i - i_col)/5)
        ax[i_row, i_col].scatter(gpm_gage_mscale[tscale][st_id]['PREC_1']['PREC_1'], gpm_gage_mscale[tscale][st_id]['PREC_1']['value'], s = 15, alpha = tscale_meta[tscale]['alpha'], c='red')
        ax[i_row, i_col].scatter(gpm_gage_mscale[tscale][st_id]['PREC_2']['PREC_2'], gpm_gage_mscale[tscale][st_id]['PREC_2']['value'], s = 15, alpha = tscale_meta[tscale]['alpha'], c='blue')
        ax[i_row, i_col].grid('major')
        
        lo_lim = min(tscale_meta[tscale]['ticks']) - tscale_meta[tscale]['ax_offset']
        up_lim = max(tscale_meta[tscale]['ticks']) - tscale_meta[tscale]['ax_offset']
        
        ax[i_row, i_col].set_xlim([lo_lim, up_lim])
        ax[i_row, i_col].set_ylim([lo_lim, up_lim])

        ax[i_row, i_col].set_xticks(tscale_meta[tscale]['ticks'])
        ax[i_row, i_col].set_yticks(tscale_meta[tscale]['ticks'])
        
        ax[i_row, i_col].set_xticklabels(tscale_meta[tscale]['ticklabels'])
        ax[i_row, i_col].set_yticklabels(tscale_meta[tscale]['ticklabels'])

        ax[i_row, i_col].tick_params(labelsize = 20)
    fig.tight_layout()
    ax[1, 0].text(-0.2, 0.35, 'GPM (mm)', transform=ax[1, 0].transAxes, fontsize=28, rotation=90)
    ax[2, 2].text(.35, -0.15, 'Gage (mm)', transform=ax[2, 2].transAxes, fontsize=28)
    
    # ax[2, 4].set_axis_off()
    fn_out = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/figures/gage_GPM_comparisons/scatter_tscale_{tscale}.jpg'.format(tscale=tscale)
    fig.savefig(fn_out, dpi=300, bbox_inches='tight')
    plt.close(fig)


In [69]:
lut_gpm_gage

,FID,X,Y,st_id,ID,Latitude,Longitude
0,4,-93.549999,42.549998,16070001,SF01,42.542620,-93.589060
1,8,-93.549999,42.449998,16070002,SF02,42.469300,-93.565450
2,8,-93.549999,42.449998,16070003,SF03,42.452960,-93.567970
3,4,-93.549999,42.549998,16070004,SF04,42.544590,-93.525270
4,8,-93.549999,42.449998,16070005,SF05,42.428570,-93.521580
5,12,-93.549999,42.349998,16070006,SF06,42.389600,-93.500130
6,5,-93.449999,42.549998,16070007,SF07,42.515010,-93.472710
7,9,-93.449999,42.449998,16070008,SF08,42.484630,-93.441490
8,9,-93.449999,42.449998,16070009,SF09,42.445560,-93.444050
9,13,-93.449999,42.349998,16070010,SF10,42.379370,-93.402930


In [54]:

# Rainfall totals summary table


# gage_p_sum = []
# for i, st_id in enumerate(lut_gpm_gage['st_id']):
#     fid = str(lut_gpm_gage['FID'][i])
#     st_id = str(st_id)
#     gage_p_sum.append((st_id, np.nansum(sf_gages[st_id]['PREC_1']), np.nansum(sf_gages[st_id]['PREC_2']), np.nansum(gpm_gage[fid]['value'])))



# GPM has less NaN values than gages -> use dates from gage 
# gage_p_sum = pd.DataFrame(gage_p_sum, columns=['st_id', 'sum_PREC_1','sum_PREC_2', 'sum_GPM'])

# test plots for soil moisture simulations vs observations

In [3]:
import pickle
with open('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gages/ARS_data/gpm_gage_sm.pickle', 'rb') as handle:
    results = pickle.load(handle)

with open('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gpm_gage_sf.pickle', 'rb') as handle:
    gpm_gage_sf = pickle.load(handle)

In [8]:
s_props = pd.read_csv('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gages/sm_gages_w_SProps.csv')
prod_list = dict({
                'tscales': ['H'],
                'scale_f': 1,
                'dt_col': 'date',
                'p_cols': ['value', 'PREC_1', 'PREC_2'],
                'p_thres': 0,
                'alias': ['gpm', 'gage1', 'gage2']
                 })

In [19]:
for i, st_id in enumerate(gpm_gage_sf['lut_gpm_gage']['st_id'][0:15]):
    fig, ax = plt.subplots( figsize = (16, 8))
    st_id = str(st_id)
    
    idx = np.where(s_props['name']=='ARS' + st_id)
    
    if len(idx[0]) ==0:
        idx = 61 
    else:
        idx = idx[0][0]

    theta_s = s_props['theta_s'][idx] # cm3/cm3
    theta_s_mm = theta_s * 1000 # cm^3/cm^3 to mm in a reference one-meter soil column
    # theta_s_mm*theta_s

    i_col = i % 5
    i_row = int((i - i_col)/5)
    # ax.plot(results[st_id]['value']['res_p'][:,3], results[st_id]['value']['res_p'][:,1],  c='red')
    # ax.plot(results[st_id]['PREC_1']['res_p'][:,3], results[st_id]['PREC_1']['res_p'][:,1], c='blue')
    theta_s_sensor = np.percentile(results[st_id]['SM']['SM_avg'], 97)
    ax.plot(results[st_id]['PREC_2']['res_p'][:,3], results[st_id]['PREC_2']['res_p'][:,1]*theta_s_sensor/theta_s, c='green')
    ax.plot(results[st_id]['SM']['date'], results[st_id]['SM']['SM_avg'],   c='red')
    # ax[i_row, i_col].plot(gpm_gage_mscale[tscale][st_id]['PREC_2']['PREC_2'], gpm_gage_mscale[tscale][st_id]['PREC_2']['value'], s = 15, alpha = tscale_meta[tscale]['alpha'], c='blue')
    # ax[i_row, i_col].grid('major')
    
    # lo_lim = min(tscale_meta[tscale]['ticks']) - tscale_meta[tscale]['ax_offset']
    # up_lim = max(tscale_meta[tscale]['ticks']) - tscale_meta[tscale]['ax_offset']
    
    # ax[i_row, i_col].set_xlim([lo_lim, up_lim])
    ax.set_ylim([-0.1, 0.6])

    # ax[i_row, i_col].set_xticks(tscale_meta[tscale]['ticks'])
    # ax[i_row, i_col].set_yticks(tscale_meta[tscale]['ticks'])
    
    # ax[i_row, i_col].set_xticklabels(tscale_meta[tscale]['ticklabels'])
    # ax[i_row, i_col].set_yticklabels(tscale_meta[tscale]['ticklabels'])
    # ax.set_xlim([datetime(2015,5,1), datetime(2015,7,1)])
    ax.tick_params(labelsize = 20)
    fig.tight_layout()
    # ax[1, 0].text(-0.2, 0.35, 'GPM (mm)', transform=ax[1, 0].transAxes, fontsize=28, rotation=90)
    # ax[2, 2].text(.35, -0.15, 'Gage (mm)', transform=ax[2, 2].transAxes, fontsize=28)

    # input()
    fn_out = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/figures/gage_GPM_comparisons/test_sm/{st_id}.jpg'.format(st_id=st_id)
    fig.savefig(fn_out, dpi=300, bbox_inches='tight')
    plt.close(fig)


In [38]:
out_fn_format = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/hydrovise_data/sm/{year}/{prod}/{st_id}.csv'

for i, st_id in enumerate(gpm_gage_sf['lut_gpm_gage']['st_id'][0:15]):
    st_id = str(st_id)
    
    idx = np.where(s_props['name']=='ARS' + st_id)
    
    if len(idx[0]) ==0:
        idx = 61 
    else:
        idx = idx[0][0]

    theta_s = s_props['theta_s'][idx] # cm3/cm3
    theta_s_mm = theta_s * 1000 # cm^3/cm^3 to mm in a reference one-meter soil column

    theta_s_sensor = np.percentile(results[st_id]['SM']['SM_avg'], 97)
    sm_obs = pd.DataFrame({'date': results[st_id]['SM']['date'], 'SM':results[st_id]['SM']['SM_avg']})
    sm_obs['date'] = pd.to_datetime(sm_obs['date'])
    sm_obs['SM'] = sm_obs['SM'].astype('float')
    for year in range(2013,2021):
                idx = sm_obs['date'].dt.year == year
                subset_obs = sm_obs.loc[idx]
                fn_out = out_fn_format.format(year = str(year), prod = 'sm_obs', st_id= st_id)
                [out_path, _]  = os.path.split(fn_out)
                if not os.path.exists(out_path):
                    os.makedirs(out_path)
                subset_obs.to_csv(fn_out,index=False,float_format='%.4f', date_format='%Y-%m-%d %H:%M')

    for i, p_col in enumerate(prod_list['p_cols']):
        date_    = results[st_id][p_col]['res_p'][:,3]
        sm_      = results[st_id][p_col]['res_p'][:,1]*theta_s_sensor/theta_s
        df_sim   =  pd.DataFrame({'date':date_, 'SM':sm_} ,dtype = object)
        df_sim['date'] = pd.to_datetime(df_sim['date'])
        df_sim['SM'] = df_sim['SM'].astype('float')

        for year in range(2013,2021):
            idx = df_sim['date'].dt.year == year
            subset = df_sim.loc[idx]
            fn_out = out_fn_format.format(year = str(year), prod = prod_list['alias'][i], st_id= st_id)
            [out_path, _]  = os.path.split(fn_out)
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            
            subset.to_csv(fn_out,index=False, float_format='%.4f', date_format='%Y-%m-%d %H:%M')



# Plot metrics

PDF of the yearly-based metrics for all gages for:
gage 1
gage 2
gpm


In [45]:
metric_list = { 'correlationcoefficient':   {'max':1,       'min':-0.1, 'label': 'Cor. Coeff. [-]',               'bw':.04,   'nbins':11, 'cmap':'YlOrRd'},
                # 'spearmann_corr':           {'max':1,       'min':-1,   'label': 'Spearman Cor. Coeff. [-]',    'bw':.04,   'nbins':10, 'cmap':'YlOrRd'},
                'kge':                      {'max':1,       'min':-0.1, 'label': 'KGE [-]',                     'bw':.04,   'nbins':11, 'cmap':'YlOrRd'},
                # 'kge_non_parametric':       {'max':1,       'min':-1,   'label': 'Non-Param. KGE [-]',          'bw':.04,   'nbins':11, 'cmap':'YlOrRd'},
                'rmse':                     {'max':0.20,    'min':0,    'label': 'RMSE [$cm^3/cm^3$]',          'bw':0.01,  'nbins':10, 'cmap':'YlOrRd'}, 
                'rrmse':                    {'max':1.0,     'min':0,    'label': 'Rel. RMSE [$cm^3/cm^3$]',     'bw':0.01,  'nbins':10, 'cmap':'YlOrRd'},
                'mae':                      {'max':0.16,    'min':0,    'label': 'MAE [$cm^3/cm^3$]',           'bw':0.01,  'nbins':8, 'cmap':'YlOrRd'}, 
                'bias':                     {'max':0.2,     'min':-0.2, 'label': 'Bias [$cm^3/cm^3$]',          'bw':0.01,  'nbins':8, 'cmap':'RdBu'}}
combs = [('gage1', 'SM'),
 ('gage2', 'SM'),
('gpm', 'SM')]
def genTitle(type1, type2):

    type1 = type1.replace('gage1', "Gage 1").replace('gage2', "Gage 2").replace('gpm', "GPM")
    type2 = type2.upper().replace('SM', "Sensor Average")
    return type1 + ' vs. ' + type2

In [46]:
metrics = pd.read_csv('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/metrics/SM_metrics.csv')

In [47]:
from sklearn.neighbors import KernelDensity
combs = [('gage1', 'SM'),
 ('gage2', 'SM'),
('gpm', 'SM')]

ln_colors = ['c','r','g', 'b', 'm', 'y', 'k', 'w']

fig_path_fmt = r'/mnt/d/ubuntu/projects/gatechProjects/StochSM/figures/metrics/'
fn_temp = '{met_name}.jpg'
n_y=2
n_x=1
n_met = 3
fig, ax = plt.subplots(n_met,n_x*n_y, figsize=(20, 15))
axis_font = {'fontname':'Arial', 'size':'20'}
# j=0    
for i, met_name in enumerate(list(metric_list.keys())):
    cm1 = plt.cm.get_cmap(metric_list[met_name]['cmap'], metric_list[met_name]['nbins']-0)
    for j, comb in enumerate(combs):
        type1, type2 = comb
        comb_name = type1 + '_' + type2
        title_str = genTitle(type1, type2)
        i_x, i_y = (int(i%n_met), int(i/(3)))
        
        # if i_y==0:
        ax[i_x, i_y].set_ylabel(r"[\%]", **axis_font)
        col = ln_colors[j]
        ix = metrics['prod']==comb_name
        data = metrics[ix]
        
        kde = KernelDensity(bandwidth=metric_list[met_name]['bw'], kernel='gaussian')
        x = data[met_name].values
        kde.fit(x[~np.isnan(x), None])
        x_d = np.linspace(metric_list[met_name]['min'], metric_list[met_name]['max'], 100)
        logprob = kde.score_samples(x_d[:, None])
        label_str = genTitle(type1, type2)
        ax[i_x, i_y].plot(x_d, np.exp(logprob), linewidth=3, label = label_str, c=col )
        ax[i_x, i_y].axvline(x = np.median(x[~np.isnan(x), None]),linewidth=3, color=col)
    ax[i_x,  i_y].set_xlabel(str(metric_list[met_name]['label']),fontsize=20 )
    ax[i_x, i_y].tick_params(labelsize=20)

fig.tight_layout()
plt.legend(fontsize=16,loc='lower center', bbox_to_anchor=(-0.02, 3.5),
          fancybox=True, shadow=True, ncol=5)

out_path = fig_path_fmt
if os.path.exists(out_path)==False:
    os.makedirs(out_path)
fn_out = os.path.join(out_path, 'all_metrics.jpg')
fig.savefig(fn_out, dpi=300, bbox_inches='tight')
plt.close(fig)

# event-based

In [2]:
import pickle
with open('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gages/ARS_data/gpm_gage_sm_event_based.pickle', 'rb') as handle:
    results = pickle.load(handle)

In [9]:
out_fn_format = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/hydrovise_data/sm_event_based/{year}/{prod}/{st_id}.csv'

for i, st_id in enumerate(gpm_gage_sf['lut_gpm_gage']['st_id'][0:15]):
    st_id = str(st_id)
    
    idx = np.where(s_props['name']=='ARS' + st_id)
    
    if len(idx[0]) ==0:
        idx = 61 
    else:
        idx = idx[0][0]

    theta_s = s_props['theta_s'][idx] # cm3/cm3
    theta_s_mm = theta_s * 1000 # cm^3/cm^3 to mm in a reference one-meter soil column

    theta_s_sensor = np.percentile(results[st_id]['SM']['SM_avg'], 97)
    sm_obs = pd.DataFrame({'date': results[st_id]['SM']['date'], 'SM':results[st_id]['SM']['SM_avg']})
    sm_obs['date'] = pd.to_datetime(sm_obs['date'])
    sm_obs['SM'] = sm_obs['SM'].astype('float')
    for year in range(2013,2021):
                idx = sm_obs['date'].dt.year == year
                subset_obs = sm_obs.loc[idx]
                fn_out = out_fn_format.format(year = str(year), prod = 'sm_obs', st_id= st_id)
                [out_path, _]  = os.path.split(fn_out)
                if not os.path.exists(out_path):
                    os.makedirs(out_path)
                subset_obs.to_csv(fn_out,index=False,float_format='%.4f', date_format='%Y-%m-%d %H:%M')

    for i, p_col in enumerate(prod_list['p_cols']):
        date_    = results[st_id][p_col]['res_p'][:,3]
        sm_      = results[st_id][p_col]['res_p'][:,1]*theta_s_sensor/theta_s
        df_sim   =  pd.DataFrame({'date':date_, 'SM':sm_} ,dtype = object)
        df_sim['date'] = pd.to_datetime(df_sim['date'])
        df_sim['SM'] = df_sim['SM'].astype('float')

        for year in range(2013,2021):
            idx = df_sim['date'].dt.year == year
            subset = df_sim.loc[idx]
            fn_out = out_fn_format.format(year = str(year), prod = prod_list['alias'][i], st_id= st_id)
            [out_path, _]  = os.path.split(fn_out)
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            
            subset.to_csv(fn_out,index=False, float_format='%.4f', date_format='%Y-%m-%d %H:%M')



In [15]:
metric_list = { 'correlationcoefficient':   {'max':1,       'min':-0.1, 'label': 'Cor. Coeff. [-]',               'bw':.04,   'nbins':11, 'cmap':'YlOrRd'},
                # 'spearmann_corr':           {'max':1,       'min':-1,   'label': 'Spearman Cor. Coeff. [-]',    'bw':.04,   'nbins':10, 'cmap':'YlOrRd'},
                'kge':                      {'max':1,       'min':-0.1, 'label': 'KGE [-]',                     'bw':.04,   'nbins':11, 'cmap':'YlOrRd'},
                # 'kge_non_parametric':       {'max':1,       'min':-1,   'label': 'Non-Param. KGE [-]',          'bw':.04,   'nbins':11, 'cmap':'YlOrRd'},
                'rmse':                     {'max':0.20,    'min':0,    'label': 'RMSE [$cm^3/cm^3$]',          'bw':0.01,  'nbins':10, 'cmap':'YlOrRd'}, 
                'rrmse':                    {'max':1.0,     'min':0,    'label': 'Rel. RMSE [$cm^3/cm^3$]',     'bw':0.01,  'nbins':10, 'cmap':'YlOrRd'},
                'mae':                      {'max':0.16,    'min':0,    'label': 'MAE [$cm^3/cm^3$]',           'bw':0.01,  'nbins':8, 'cmap':'YlOrRd'}, 
                'bias':                     {'max':0.2,     'min':-0.2, 'label': 'Bias [$cm^3/cm^3$]',          'bw':0.01,  'nbins':8, 'cmap':'RdBu'}}
combs = [('gage1', 'SM'),
 ('gage2', 'SM'),
('gpm', 'SM')]
def genTitle(type1, type2):

    type1 = type1.replace('gage1', "Gage 1").replace('gage2', "Gage 2").replace('gpm', "GPM")
    type2 = type2.upper().replace('SM', "Sensor Average")
    return type1 + ' vs. ' + type2

metrics = pd.read_csv('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/metrics/SM_metrics_event_based.csv')

In [16]:
from sklearn.neighbors import KernelDensity
combs = [('gage1', 'SM'),
 ('gage2', 'SM'),
('gpm', 'SM')]

ln_colors = ['c','r','g', 'b', 'm', 'y', 'k', 'w']

fig_path_fmt = r'/mnt/d/ubuntu/projects/gatechProjects/StochSM/figures/metrics/'
fn_temp = '{met_name}.jpg'
n_y=2
n_x=1
n_met = 3
fig, ax = plt.subplots(n_met,n_x*n_y, figsize=(20, 15))
axis_font = {'fontname':'Arial', 'size':'20'}
# j=0    
for i, met_name in enumerate(list(metric_list.keys())):
    cm1 = plt.cm.get_cmap(metric_list[met_name]['cmap'], metric_list[met_name]['nbins']-0)
    for j, comb in enumerate(combs):
        type1, type2 = comb
        comb_name = type1 + '_' + type2
        title_str = genTitle(type1, type2)
        i_x, i_y = (int(i%n_met), int(i/(3)))
        
        # if i_y==0:
        ax[i_x, i_y].set_ylabel(r"[\%]", **axis_font)
        col = ln_colors[j]
        ix = metrics['prod']==comb_name
        data = metrics[ix]
        
        kde = KernelDensity(bandwidth=metric_list[met_name]['bw'], kernel='gaussian')
        x = data[met_name].values
        kde.fit(x[~np.isnan(x), None])
        x_d = np.linspace(metric_list[met_name]['min'], metric_list[met_name]['max'], 100)
        logprob = kde.score_samples(x_d[:, None])
        label_str = genTitle(type1, type2)
        ax[i_x, i_y].plot(x_d, np.exp(logprob), linewidth=3, label = label_str, c=col )
        ax[i_x, i_y].axvline(x = np.median(x[~np.isnan(x), None]),linewidth=3, color=col)
    ax[i_x,  i_y].set_xlabel(str(metric_list[met_name]['label']),fontsize=20 )
    ax[i_x, i_y].tick_params(labelsize=20)

fig.tight_layout()
plt.legend(fontsize=16,loc='lower center', bbox_to_anchor=(-0.02, 3.5),
          fancybox=True, shadow=True, ncol=5)

out_path = fig_path_fmt
if os.path.exists(out_path)==False:
    os.makedirs(out_path)
fn_out = os.path.join(out_path, 'all_metrics_event_based.jpg')
fig.savefig(fn_out, dpi=300, bbox_inches='tight')
plt.close(fig)

In [17]:
with open('/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/gages/ARS_data/gpm_gage_sm_event_based.pickle', 'rb') as handle:
    results = pickle.load(handle)

In [18]:
out_fn_format = '/mnt/d/ubuntu/projects/gatechProjects/StochSM/data/hydrovise_data/sm_event_based/{year}/{prod}/{st_id}.csv'

for i, st_id in enumerate(gpm_gage_sf['lut_gpm_gage']['st_id'][0:15]):
    st_id = str(st_id)
    
    idx = np.where(s_props['name']=='ARS' + st_id)
    
    if len(idx[0]) ==0:
        idx = 61 
    else:
        idx = idx[0][0]

    theta_s = s_props['theta_s'][idx] # cm3/cm3
    theta_s_mm = theta_s * 1000 # cm^3/cm^3 to mm in a reference one-meter soil column

    theta_s_sensor = np.percentile(results[st_id]['SM']['SM_avg'], 97)
    sm_obs = pd.DataFrame({'date': results[st_id]['SM']['date'], 'SM':results[st_id]['SM']['SM_avg']})
    sm_obs['date'] = pd.to_datetime(sm_obs['date'])
    sm_obs['SM'] = sm_obs['SM'].astype('float')
    for year in range(2013,2021):
                idx = sm_obs['date'].dt.year == year
                subset_obs = sm_obs.loc[idx]
                fn_out = out_fn_format.format(year = str(year), prod = 'sm_obs', st_id= st_id)
                [out_path, _]  = os.path.split(fn_out)
                if not os.path.exists(out_path):
                    os.makedirs(out_path)
                subset_obs.to_csv(fn_out,index=False,float_format='%.4f', date_format='%Y-%m-%d %H:%M')

    for i, p_col in enumerate(prod_list['p_cols']):
        date_    = results[st_id][p_col]['res_p'][:,3]
        sm_      = results[st_id][p_col]['res_p'][:,1]*theta_s_sensor/theta_s
        df_sim   =  pd.DataFrame({'date':date_, 'SM':sm_} ,dtype = object)
        df_sim['date'] = pd.to_datetime(df_sim['date'])
        df_sim['SM'] = df_sim['SM'].astype('float')

        for year in range(2013,2021):
            idx = df_sim['date'].dt.year == year
            subset = df_sim.loc[idx]
            fn_out = out_fn_format.format(year = str(year), prod = prod_list['alias'][i], st_id= st_id)
            [out_path, _]  = os.path.split(fn_out)
            if not os.path.exists(out_path):
                os.makedirs(out_path)
            
            subset.to_csv(fn_out,index=False, float_format='%.4f', date_format='%Y-%m-%d %H:%M')

